<a href="https://colab.research.google.com/github/nimishwork6699/Deep_Learning_Pytorch/blob/main/CNN.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import pandas as pd
from sklearn.model_selection import train_test_split
import torch
import torch.nn as nn
from torch.utils.data import Dataset,DataLoader
import torch.optim as optim
import matplotlib.pyplot as plt

In [2]:
# Set manual seed
torch.manual_seed(42)

In [3]:
# check gpu availiblity
device=torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f"Using device :{device}")

Using device :cuda


In [4]:
import kagglehub

# Download latest version
path = kagglehub.dataset_download("zalando-research/fashionmnist")

print("Path to dataset files:", path)

Using Colab cache for faster access to the 'fashionmnist' dataset.
Path to dataset files: /kaggle/input/fashionmnist


In [5]:
df = pd.read_csv(path + "/fashion-mnist_train.csv")

In [6]:
# train test split
X=df.iloc[:,1:].values
y=df.iloc[:,0].values

In [7]:
X_train,X_test,y_train,y_test=train_test_split(X,y,test_size=0.2,random_state=42)

In [8]:
# scaling the features
X_train=X_train/255.0
X_test=X_test/255.0

In [9]:
X_train

array([[0.        , 0.        , 0.        , ..., 0.        , 0.        ,
        0.        ],
       [0.        , 0.        , 0.        , ..., 0.        , 0.        ,
        0.        ],
       [0.        , 0.        , 0.        , ..., 0.        , 0.        ,
        0.        ],
       ...,
       [0.        , 0.        , 0.        , ..., 0.        , 0.        ,
        0.        ],
       [0.        , 0.        , 0.        , ..., 0.        , 0.        ,
        0.        ],
       [0.        , 0.        , 0.        , ..., 0.00392157, 0.        ,
        0.        ]])

In [10]:
class MyCoustomDataset(Dataset):

  def __init__(self,feature,labels):
    self.feature=torch.tensor(feature,dtype=torch.float32).reshape(-1,1,28,28)
    self.labels=torch.tensor(labels,dtype=torch.long)

  def __len__(self):

    return len(self.feature)

  def __getitem__(self, index):
    return self.feature[index],self.labels[index]


In [11]:
# creating train_dataset
# features and labels ki jagah we are making it as a subsitute
train_dataset=MyCoustomDataset(X_train,y_train)

In [12]:
# creating test dataset
test_dataset=MyCoustomDataset(X_test,y_test)

In [13]:
# creating train and test loader
train_loader=DataLoader(train_dataset,batch_size=32,shuffle=True,pin_memory=True)
test_loader=DataLoader(test_dataset,batch_size=32,shuffle=False,pin_memory=True)

In [21]:
from torch.nn.modules.dropout import Dropout
from torch.nn.modules.activation import ReLU
from torch.nn.modules.linear import Linear
from torch.nn.modules.batchnorm import BatchNorm2d
from torch.nn.modules.pooling import MaxPool2d
# define NN Module
class MyNN(nn.Module):
  def __init__(self, input_features):

    super().__init__()
    self.features=nn.Sequential(
        nn.Conv2d(input_features,32,kernel_size=3,padding='same'),
        nn.ReLU(),
        nn.BatchNorm2d(32),
        nn.MaxPool2d(kernel_size=2,stride=2),

        nn.Conv2d(32,64,kernel_size=3,padding='same'),
        nn.ReLU(),
        nn.BatchNorm2d(64),
        nn.MaxPool2d(kernel_size=2,stride=2),
    )
    self.classifier=nn.Sequential(
        nn.Flatten(),
        nn.Linear(64*7*7,128),
        nn.ReLU(),
        nn.Dropout(p=0.4),

        nn.Linear(128,64),
        nn.ReLU(),
        nn.Dropout(p=0.4),

        nn.Linear(64,10),
    )

  def forward(self,X):
    X=self.features(X)
    X=self.classifier(X)

    return X

In [22]:
# set learning rate and epochs
epochs = 100
learning_rate = 0.01

In [23]:
# instatiate the model
model = MyNN(1)
model=model.to(device)

# loss function
criterion = nn.CrossEntropyLoss()

# optimizer
optimizer = optim.SGD(model.parameters(), lr= learning_rate,weight_decay=1e-4)

In [24]:
# training loop

for epoch in range(epochs):

  total_epoch_loss = 0

  for batch_features, batch_labels in train_loader:

    # moving data to gpu
    batch_features, batch_labels=batch_features.to(device),batch_labels.to(device)

    # forward pass
    outputs = model(batch_features)

    # calculate loss
    loss = criterion(outputs, batch_labels)

    # back pass
    optimizer.zero_grad()
    loss.backward()

    # update grads
    optimizer.step()

    total_epoch_loss = total_epoch_loss + loss.item()

  avg_loss = total_epoch_loss/len(train_loader)
  print(f'Epoch: {epoch + 1} , Loss: {avg_loss}')


Epoch: 1 , Loss: 0.6651311286985874
Epoch: 2 , Loss: 0.38902034710596006
Epoch: 3 , Loss: 0.32666904942691327
Epoch: 4 , Loss: 0.292796078513066
Epoch: 5 , Loss: 0.26611618318408725
Epoch: 6 , Loss: 0.24526808562378088
Epoch: 7 , Loss: 0.22998879921560486
Epoch: 8 , Loss: 0.21127169035002588
Epoch: 9 , Loss: 0.19989039552335938
Epoch: 10 , Loss: 0.18937079885664085
Epoch: 11 , Loss: 0.17649324431146185
Epoch: 12 , Loss: 0.16550071814035375
Epoch: 13 , Loss: 0.1561500213323161
Epoch: 14 , Loss: 0.148152059089082
Epoch: 15 , Loss: 0.13944503308770556
Epoch: 16 , Loss: 0.13446118116906533
Epoch: 17 , Loss: 0.12696557740711917
Epoch: 18 , Loss: 0.12041514638097336
Epoch: 19 , Loss: 0.11728912937501446
Epoch: 20 , Loss: 0.10736645227313663
Epoch: 21 , Loss: 0.10124546415824442
Epoch: 22 , Loss: 0.09802319698982562
Epoch: 23 , Loss: 0.09142845207437252
Epoch: 24 , Loss: 0.08922867941359679
Epoch: 25 , Loss: 0.08526950164837763
Epoch: 26 , Loss: 0.08052150350847902
Epoch: 27 , Loss: 0.0773679

In [25]:
# evaluation code
total = 0
correct = 0

with torch.no_grad():

  for batch_features, batch_labels in test_loader:
     # moving data to gpu
    batch_features, batch_labels=batch_features.to(device),batch_labels.to(device)

    outputs = model(batch_features)

    _, predicted = torch.max(outputs, 1)

    total = total + batch_labels.shape[0]

    correct = correct + (predicted == batch_labels).sum().item()

print(correct/total)


0.9128333333333334
